# Impact des ZFE sur le NO₂ à Grenoble et Paris  
Par Pierre-Andréa Silvente & Paolo Maunas – groupe X

<a id="introduction"></a>
## Introduction

Les zones à faibles émissions (ZFE) sont devenues un instrument central des politiques de lutte contre la pollution atmosphérique dans les grandes agglomérations françaises. Elles reposent sur des restrictions de circulation fondées sur les vignettes Crit’Air, avec pour objectif de réduire les émissions liées au trafic routier, en particulier pour le dioxyde d’azote (NO₂).

Mesurer l’effet réel de ces dispositifs n’est cependant pas immédiat. Les concentrations de NO₂ évoluent sous l’effet de nombreux facteurs : tendance de fond liée au renouvellement du parc automobile, conditions météorologiques, réorganisation du trafic, chocs exceptionnels comme la crise sanitaire de 2020, ou encore d’autres politiques publiques environnementales. Une simple comparaison "avant / après" l’introduction d’une ZFE ne permet donc pas d’attribuer de façon crédible une éventuelle baisse du NO₂ au seul dispositif.

Dans ce projet, nous cherchons à quantifier dans quelle mesure la mise en place des ZFE à Grenoble et à Paris s’est traduite par une diminution des concentrations de NO₂ au niveau de certaines stations urbaines. (Le choix de ces deux villes n'est pas anodin ! On compare les effets de cette mesure dans une très grande ville et une ville un peu plus modeste) Pour cela, nous combinons plusieurs jeux de données open data :

- des données géographiques décrivant les périmètres de ZFE et leurs dates de mise en œuvre ;
- des séries journalières de NO₂ pour des stations situées à l’intérieur des ZFE de Grenoble et de Paris ;
- des séries comparables pour des stations d’autres grandes villes françaises, utilisées comme unités "donneuses" où cette fois-ci aucune ZFE n'a été mise en place.

Nous nous concentrons sur le NO₂ parmi les polluants disponibles sur les plateformes de surveillance (CO, benzène C₆H₆, NOₓ, O₃, SO₂, PM₁₀, etc.), car il s’agit de l’indicateur réglementaire le plus directement lié au trafic routier et celui pour lequel les dépassements des valeurs limites ont motivé la mise en place des ZFE. C’est aussi le polluant pour lequel la couverture spatiale et temporelle des stations est la plus homogène, ce qui facilite les comparaisons entre villes et la construction de contrefactuels.

La démarche repose sur trois blocs :

1. une analyse descriptive détaillée des niveaux de NO₂ avant et après la mise en place des ZFE, à Grenoble et à Paris, et leur comparaison avec les villes donneuses ;
2. un modèle de contrôle synthétique (avec pénalisation Ridge, Lasso et ElasticNet) pour construire, pour chaque station traitée, un contrefactuel "sans ZFE" à partir des donneurs ;
3. des modèles de machine learning (forêt aléatoire, gradient boosting) utilisés comme contrefactuels alternatifs.

L’objectif, ambitieux, est d'étudier la possibilité d'un impact causal des ZFE sur la réduction de NO₂ dans l'atmosphère, tout en proposant une analyse structurée, reproductible et transparente à partir de données publiques, et en mettant en évidence les ordres de grandeur plausibles des effets et les limites de l’approche.


## Sommaire

- [Installation](#installation)
- [Préparation des données](#preparation-des-donnees)
  - [Adresses](#adresses)
  - [Données ZFE](#donnees-zfe)
  - [Données de pollution à Grenoble](#donnees-grenoble)
  - [Données de pollution à Paris](#donnees-paris)
  - [Donneurs : sélection des villes et des stations](#donneurs)
  - [Construction des séries journalières et mensuelles](#series)
  - [Gestion des valeurs manquantes](#valeurs-manquantes)
- [Analyse descriptive](#analyse-descriptive)
  - [Grenoble : stations Les Frênes et Boulevards](#desc-grenoble)
  - [Paris : stations Champs-Élysées et Les Halles](#desc-paris)
  - [Comparaison avec les villes donneuses](#desc-donneurs)
- [Modélisation par contrôle synthétique](#scm)
  - [Principe et mise en œuvre](#scm-principe)
  - [Résultats pour Grenoble](#scm-grenoble)
  - [Résultats pour Paris](#scm-paris)
  - [Effets moyens du traitement (ATT)](#scm-att)
- [Modélisation par méthodes de machine learning](#ml)
  - [Spécification et stratégie d’estimation](#ml-spec)
  - [Résultats pour Grenoble](#ml-grenoble)
  - [Résultats pour Paris](#ml-paris)
- [Discussion et limites](#discussion)
- [Conclusion et perspectives](#conclusion)


<a id="installation"></a>
## Installation

Ce notebook suppose que le dépôt Git a été cloné en local et que les fichiers de données nettoyés sont disponibles dans le dossier `data/` à la racine du projet.

Les fonctions génériques de préparation des données et de modélisation sont regroupées dans des scripts Python du dossier `scripts/` (`data_prep.py`, `scm_models.py`, `ml_models.py`). Le notebook importe ces fonctions au démarrage afin de limiter la duplication de code et de rendre la structure du projet plus lisible.

Pour reproduire ce rapport à partir d’un environnement vierge :

1. Cloner le dépôt GitHub.  
2. Créer un environnement virtuel (optionnel mais recommandé) et l’activer.  
3. Installer les dépendances listées dans `requirements.txt` avec `pip install -r requirements.txt`.  
4. Exécuter ce notebook depuis le dossier `zfe-scm/`, en veillant à ce que le dossier `data/` et le dossier `scripts/` soient présents à la racine du projet.


In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.grid"] = True

PROJECT_ROOT = Path().resolve()
DATA_DIR = PROJECT_ROOT / "data"
SCRIPTS_DIR = PROJECT_ROOT / "scripts"

print("Racine du projet :", PROJECT_ROOT)
print("Dossier data     :", DATA_DIR)
print("Dossier scripts  :", SCRIPTS_DIR)

if not DATA_DIR.exists():
    raise FileNotFoundError(f"Le dossier data est introuvable : {DATA_DIR}")

if not SCRIPTS_DIR.exists():
    raise FileNotFoundError(f"Le dossier scripts est introuvable : {SCRIPTS_DIR}")

from scripts.data_prep import build_monthly_series, summarize_missing_daily
from scripts.scm_models import fit_penalized_scm, compute_att_summary
from scripts.zfe_data import build_aires_voies_flat, build_zfe_clean_tables
from scripts.build_pollution_data import build_no2_daily_for_zfe





Racine du projet : C:\Users\Pierre\Desktop\Projet Python pour la Data Science
Dossier data     : C:\Users\Pierre\Desktop\Projet Python pour la Data Science\data
Dossier scripts  : C:\Users\Pierre\Desktop\Projet Python pour la Data Science\scripts


<a id="preparation-des-donnees"></a>
## Préparation des données

<a id="adresses"></a>
### 1. Adresses

Toutes les données utilisées dans ce projet proviennent de sources ouvertes. Pour assurer la reproductibilité, nous listons ici les principales URL d’origine (portails open data, export ATMO, etc.). Nous utiliserons une copie locale des fichiers bruts le dossier `data/`.

Les liens exacts vers les exports utilisés sont renseignés dans les variables ci-dessous. Ils ne sont pas exploités directement dans ce notebook, qui travaille à partir des fichiers bruts, mais ils documentent la provenance des données.

In [2]:
# Adresses des principales sources de données

ZFE_GEOJSON_AIRES_URL = "https://transport.data.gouv.fr/resources/79567"
ZFE_GEOJSON_VOIES_URL = "https://transport.data.gouv.fr/resources/79568"
ZFE_IDENTIFIANTS_URL = "https://transport.data.gouv.fr/resources/79760"  

MESURES_NO2_RAW_URL = "https://www.geodair.fr/donnees/export-advanced"


<a id="donnees-zfe"></a>
### 2. Données ZFE

Les périmètres de ZFE utilisés dans ce projet proviennent des fichiers GeoJSON publiés sur data.gouv et sont `aires.geojson` qui décrit les aires sur lesquelles la ZFE s’applique (polygones géographiques) ainsi que `voies.geojson` qui décrit les tronçons de voies concernés.

Chaque feature contient deux blocs d’information : un bloc `properties` avec des attributs géographiques et réglementaires et un bloc `publisher` qui décrit la ZFE et la collectivité (identifiant, nom, SIREN, forme juridique, etc.).

La table `zfe_ids.csv` également récupérée sur data.gouv fournit des informations supplémentaires par ZFE (SIREN, structure juridique, etc.).

Pour rendre ces données exploitables dans le reste du projet, nous avons mis en place le pipeline suivant, implémenté dans le script `scripts/zfe_data.py` :

1. **Aplatissement des GeoJSON**  
   Les fichiers `aires.geojson` et `voies.geojson` sont lus puis aplatis via la fonction `flatten_geojson`.  
   Chaque feature est transformée en une ligne, avec :
   - les champs de `properties` ;
   - les champs de `publisher` préfixés en `publisher_`.  
   Nous obtenons deux tables intermédiaires : `aires_flat.csv` et `voies_flat.csv`.

2. **Construction de tables nettoyées**  
   À partir de ces tables aplaties, nous sélectionnons des colonnes qui pourraient nous être pertinentes pour la suite de notre analyse. Donc : 
   - pour les aires : identifiant de ZFE, SIREN et nom de la collectivité, dates de début et de fin, informations sur les restrictions par type de véhicule (VP, VUL, poids lourds, bus, deux-roues), liens vers les arrêtés et sites d’information ;
   - pour les voies : mêmes informations ZFE/collectivité, plus les identifiants de tronçon (`osm_id`, `ref`, sens de circulation, indicateurs de dérogation, etc.).  
   Les dates de début et de fin sont converties au format `datetime`. Enfin, les résultats sont sauvegardés dans deux fichiers csv : `aires_clean.csv` et `voies_clean.csv`.

3. **Table de métadonnées `zfe_meta`**  
   À partir d’`aires_clean`, nous construisons une table de synthèse avec une ligne par ZFE (`publisher_zfe_id`, `publisher_siren`, `publisher_nom`).  
   Pour chaque ZFE, nous agrégons selon le processus suivant :
   - la première date de début observée dans les aires (`first_date_debut`) ;
   - la dernière date de début (`last_date_debut`) ;
   - la première date de fin si une fin est renseignée (`first_date_fin`) ;
   - le nombre d’aires couvertes (`n_aires`) ;
   - un indicateur de présence d’une restriction VP (`has_vp_restriction`).  

   Cette table est ensuite enrichie par jointure avec `zfe_ids.csv` sur le SIREN, afin de récupérer notamment la forme juridique et les informations complémentaires sur la collectivité. Le résultat final est sauvegardé sous le nom `zfe_meta.csv` et sert de table de référence pour identifier les ZFE de Grenoble et de Paris et leurs dates de mise en œuvre.

In [3]:
# Construction (ou reconstruction) de aires_flat et voies_flat à partir des GeoJSON
aires_flat, voies_flat = build_aires_voies_flat(
    data_dir=DATA_DIR,
    aires_geojson_name="aires.geojson",
    voies_geojson_name="voies.geojson",
    aires_flat_name="aires_flat.csv",
    voies_flat_name="voies_flat.csv",
)

print("aires_flat :", aires_flat.shape)
print("voies_flat :", voies_flat.shape)

# Construction des tables nettoyées et de zfe_meta
aires_clean, voies_clean, zfe_meta = build_zfe_clean_tables(
    data_dir=DATA_DIR,
    aires_flat_name="aires_flat.csv",
    voies_flat_name="voies_flat.csv",
    zfe_ids_name="zfe_ids.csv",
    aires_clean_name="aires_clean.csv",
    voies_clean_name="voies_clean.csv",
    zfe_meta_name="zfe_meta.csv",
)

print("aires_clean :", aires_clean.shape)
print("voies_clean :", voies_clean.shape)
print("zfe_meta    :", zfe_meta.shape)
print("Colonnes de zfe_meta :")
print(list(zfe_meta.columns))

zfe_meta.head(5)

aires_flat : (37, 47)
voies_flat : (8072, 32)
aires_clean : (37, 19)
voies_clean : (8072, 24)
zfe_meta    : (19, 13)
Colonnes de zfe_meta :
['publisher_zfe_id', 'publisher_siren', 'publisher_nom', 'first_date_debut', 'last_date_debut', 'first_date_fin', 'n_aires', 'has_vp_restriction', 'siren', 'code', 'epci_principal', 'autres_siren', 'forme_juridique']


,publisher_zfe_id,publisher_siren,publisher_nom,first_date_debut,last_date_debut,first_date_fin,n_aires,has_vp_restriction,siren,code,epci_principal,autres_siren,forme_juridique
0,ANGERS,244900015,CU Angers Loire Métropole,2025-01-01,2025-01-01,2026-12-31,1,True,244900015.0,ANGERS,Angers Loire Métropole,NaN,Métropole
1,ANNECY,200066793,CA du Grand Annecy,2025-01-01,2030-01-01,2027-12-31,4,True,200066793.0,ANNECY,Grand Annecy,NaN,Communauté d'agglomération
2,ANNEMASSE,200011773,CA Annemasse-Les Voirons-Agglomération,2025-01-01,2025-01-01,NaT,1,True,200011773.0,ANNEMASSE,Annemasse agglo,NaN,Communauté d'agglomération
3,BORDEAUX,243300316,Bordeaux Métropole,2025-01-01,2025-01-01,NaT,1,True,243300316.0,BORDEAUX,Bordeaux Métropole,NaN,Métropole
4,CLERMONT-FERRAND,246300701,Clermont Auvergne métropole,2023-07-01,2023-07-01,NaT,1,False,246300701.0,CLERMONT-FERRAND,Clermont Auvergne métropole,NaN,Métropole


In [4]:
filtre = zfe_meta[zfe_meta["publisher_zfe_id"].isin(["GRENOBLE", "PARIS"])]
filtre  

,publisher_zfe_id,publisher_siren,publisher_nom,first_date_debut,last_date_debut,first_date_fin,n_aires,has_vp_restriction,siren,code,epci_principal,autres_siren,forme_juridique
6,GRENOBLE,253800825,Syndicat Mixte des Mobilités de l'Aire Grenobl...,2019-05-02,2019-05-02,NaT,1,False,253800825.0,GRENOBLE,Grenoble Alpes métropole,NaN,Métropole
12,PARIS,217500016,Ville de Paris,2021-06-01,2021-06-01,NaT,1,True,217500016.0,PARIS,Ville de Paris,NaN,Autre collectivité territoriale


<a id="donnees-grenoble"></a>
### 3. Données de pollution à Grenoble

Pour Grenoble, nous utilisons l'export brut fourni par ATMO Auvergne–Rhône-Alpes, couvrant 2 stations que nous avons sélectionnés car situées dans la ZFE. Le fichier contient notamment : une colonne d’horodatage (`Date de début`) pour dater le début du prélèvement qu'on a choisi, le code et le nom de la station, le type d’implantation (urbaine, trafic, fond, etc.), le type d’influence, les coordonnées géographiques (latitude, longitude) et les concentrations moyennes par jour en NO₂, notre variable cible. Nous avons couvert les dates du 2 mai 2016 au 2 mai 2024 puisque pour le contrôle synthétique il faut un maximum de données avant la date d'intervention (qui a eu lieu en 2019 pour Grenoble, d'après le tableau ci-dessus).

Le traitement de cet export est implémenté dans la fonction `build_grenoble_no2_daily` du module `scripts/build_pollution_data.py`. Cette fonction procède comme suit : 

1. tout d'abord, elle lit le fichier brut `Export Moy. journalière - 20251204215149 - 2016-02-05 00_00 - 2024-02-05 21_00.csv`;
2. puis elle construit un jeu de données journalier au format interne du projet, avec les colonnes  
   `date`, `station_id`, `station_name`, `station_env`, `station_influence`, `no2_ug_m3`, `lat`, `lon`, ainsi qu’une étiquette de zone (`GRENOBLE`) et sauvegarde le fichier `pollution_grenoble_no2_daily_clean.csv` dans le dossier `data/`.
3. enfin, à partir d’`aires.geojson`, elle charge la géométrie de la ZFE Grenoble et permet un contrôle, pour chaque station, si elle est située ou non dans le périmètre de la ZFE. *PS : Bien sûr que ce point a été contrôlé lors du choix des stations à Grenoble, mais rien vaut mieux qu'une cross validation ;)*

Nous conserverons ce fichier comme base pour les analyses descriptives et les modèles sur Grenoble qui contient nos deux stations d’intérêt qui sont *Les Frênes* et *Grenoble Boulevards*.


In [5]:
GRENOBLE_RAW_CSV = "Export Moy. journalière - 20251204215149 - 2016-02-05 00_00 - 2024-02-05 21_00.csv"

grenoble_no2_daily, grenoble_stations_meta = build_no2_daily_for_zfe(
    data_dir=DATA_DIR,
    raw_csv_name=GRENOBLE_RAW_CSV,
    zfe_id="GRENOBLE",
    out_daily_name="pollution_grenoble_no2_daily_clean.csv",
    in_zfe_col="in_zfe_grenoble",
)

print("NO2 Grenoble (journalier, toutes stations) :", grenoble_no2_daily.shape)
display(grenoble_no2_daily.head())

print("Stations Grenoble et appartenance à la ZFE :")
display(grenoble_stations_meta)

NO2 Grenoble (journalier, toutes stations) : (5836, 9)


,date,station_id,station_name,station_env,station_influence,no2_ug_m3,lat,lon,zone
0,2016-02-05,FR15043,Grenoble Les Frenes,Urbaine,Fond,27.0,45.1619,5.7356,GRENOBLE
1,2016-02-06,FR15043,Grenoble Les Frenes,Urbaine,Fond,17.0,45.1619,5.7356,GRENOBLE
2,2016-02-07,FR15043,Grenoble Les Frenes,Urbaine,Fond,15.0,45.1619,5.7356,GRENOBLE
3,2016-02-08,FR15043,Grenoble Les Frenes,Urbaine,Fond,26.0,45.1619,5.7356,GRENOBLE
4,2016-02-09,FR15043,Grenoble Les Frenes,Urbaine,Fond,15.0,45.1619,5.7356,GRENOBLE


Stations Grenoble et appartenance à la ZFE :


,station_id,station_name,lat,lon,in_zfe_grenoble
0,FR15043,Grenoble Les Frenes,45.16190,5.735600,True
1,FR15046,Grenoble Boulevards,45.18069,5.720625,True


<a id="donnees-paris"></a>
### 4. Données de pollution à Paris

Pour Paris, nous utilisons un export brut du réseau Airparif récupéré sur le site geodair, couvrant plusieurs stations et polluants sur la période 2017-2025. La structure du fichier est similaire à celle de Grenoble : date de début de mesure, identifiant et nom de station, types d’implantation et d’influence, coordonnées géographiques, concentrations par polluant.

Le traitement est implémenté dans la fonction `build_paris_no2_daily` du module `scripts/build_pollution_data.py`. Cette fonction procède comme celle pour Grenoble : 

1. elle lit le fichier `Export Moy. journalière - 20251205114649 - 2017-08-17 00_00 - 2025-12-04 11_00.csv`
2. construit un jeu de données journalier avec les colonnes  
   `date`, `station_id`, `station_name`, `station_env`, `station_influence`, `no2_ug_m3`, `lat`, `lon` et sauvegarde `pollution_paris_no2_daily_clean.csv` qui contient ls données NO₂ journalières pour les stations parisiennes.
3. enfin, à partir d’`aires.geojson`, elle charge la géométrie de la ZFE Paris et calcule, pour chaque station, un indicateur d’appartenance au périmètre (`in_zfe_paris`) (encore une fois ça a déjà été vérifié lors du choix des stations)

Ce fichier servira de base aux analyses descriptives et à la construction du synthétique pour Paris.

In [6]:
PARIS_RAW_CSV = "Export Moy. journalière - 20251205114649 - 2017-08-17 00_00 - 2025-12-04 11_00.csv"

paris_no2_daily, paris_stations_meta = build_no2_daily_for_zfe(
    data_dir=DATA_DIR,
    raw_csv_name=PARIS_RAW_CSV,
    zfe_id="PARIS",
    out_daily_name="pollution_paris_no2_daily_clean.csv",
    in_zfe_col="in_zfe_paris",
)

print("NO2 Paris (journalier) :", paris_no2_daily.shape)
display(paris_no2_daily.head())

print("Stations Paris et appartenance à la ZFE :")
display(paris_stations_meta)

NO2 Paris (journalier) : (5268, 9)


,date,station_id,station_name,station_env,station_influence,no2_ug_m3,lat,lon,zone
0,2017-08-17,FR04031,Av Champs Elysees,Urbaine,Trafic,27.0,48.868772,2.311231,PARIS
1,2017-08-18,FR04031,Av Champs Elysees,Urbaine,Trafic,38.0,48.868772,2.311231,PARIS
2,2017-08-19,FR04031,Av Champs Elysees,Urbaine,Trafic,32.0,48.868772,2.311231,PARIS
3,2017-08-20,FR04031,Av Champs Elysees,Urbaine,Trafic,32.0,48.868772,2.311231,PARIS
4,2017-08-21,FR04031,Av Champs Elysees,Urbaine,Trafic,37.0,48.868772,2.311231,PARIS


Stations Paris et appartenance à la ZFE :


,station_id,station_name,lat,lon,in_zfe_paris
0,FR04031,Av Champs Elysees,48.868772,2.311231,True
1,FR04055,PARIS 1er Les Halles,48.862120,2.344616,True
